In [1]:
import numpy as np
import pandas as pd
import os

import cv2

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.models import load_model

In [3]:
def predict_images(path):

    model=load_model("./drowiness_new6.h5")
    results={0:'yawn',1:'no_yawn', 2:'Closed', 3: 'Open'}
    GR_dict={0:(0,0,255),1:(0,255,0), 2:(0,0,255), 3:(0,255,0)}
    im = cv2.imread(path)
    rect_size = 4

    haarcascade = cv2.CascadeClassifier('prediction_images/haarcascade.xml')
    haarcascade2 =  cv2.CascadeClassifier('prediction_images/haarcascade_frontalface_default.xml')
    im=cv2.flip(im,1,1)

    rerect_size = cv2.resize(im, ((im.shape[1] // rect_size, im.shape[0] // rect_size)))
    faces = haarcascade2.detectMultiScale(rerect_size)

    while True:
        for f in faces:
            (x, y, w, h) = [v * rect_size for v in f] 
            
            face_img = im[y:y+h, x:x+w]
            rerect_sized=cv2.resize(face_img,(145,145))
            normalized=rerect_sized/255.0
            reshaped=np.reshape(normalized,(1,145,145,3))
            reshaped = np.vstack([reshaped])
            result=model.predict(reshaped)

            
            label=np.argmax(result,axis=1)[0]
        
            cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
            cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
            cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
            cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
            cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

        cv2.imshow('LIVE',   im)
        key = cv2.waitKey(10)

        if key == 27: 
            break

    cv2.destroyAllWindows()

In [4]:
predict_images('prediction_images/8.jpeg')

AttributeError: 'NoneType' object has no attribute 'shape'